In [53]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [2]:
# Read data without the index column
data = pd.read_csv("heart_failure.csv", index_col=[0])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 298
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    object 
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    object 
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    object 
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    object 
 10  smoking                   299 non-null    object 
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
 13  death_event               299 non-null    object 
dtypes: float64(3), 

In [3]:
# Data statistics
data.describe(include="all")

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,death_event
count,299.000000,299,299.000000,299,299.000000,299,299.000000,299.00000,299.000000,299,299,299.000000,299.00000,299
unique,NaN,2,NaN,2,NaN,2,NaN,NaN,NaN,2,2,NaN,NaN,2
top,NaN,no,NaN,no,NaN,no,NaN,NaN,NaN,yes,no,NaN,NaN,no
freq,NaN,170,NaN,174,NaN,194,NaN,NaN,NaN,194,203,NaN,NaN,203
mean,60.833893,NaN,581.839465,NaN,38.083612,NaN,263358.029264,1.39388,136.625418,NaN,NaN,130.260870,0.32107,NaN
std,11.894809,NaN,970.287881,NaN,11.834841,NaN,97804.236869,1.03451,4.412477,NaN,NaN,77.614208,0.46767,NaN
min,40.000000,NaN,23.000000,NaN,14.000000,NaN,25100.000000,0.50000,113.000000,NaN,NaN,4.000000,0.00000,NaN
25%,51.000000,NaN,116.500000,NaN,30.000000,NaN,212500.000000,0.90000,134.000000,NaN,NaN,73.000000,0.00000,NaN
50%,60.000000,NaN,250.000000,NaN,38.000000,NaN,262000.000000,1.10000,137.000000,NaN,NaN,115.000000,0.00000,NaN
75%,70.000000,NaN,582.000000,NaN,45.000000,NaN,303500.000000,1.40000,140.000000,NaN,NaN,203.000000,1.00000,NaN


In [4]:
# Check distribution of the death_event
Counter(data["death_event"])

Counter({'no': 203, 'yes': 96})

In [18]:
# Create the labels and features
y = data["death_event"]
x = data.drop(columns=["DEATH_EVENT", "death_event"])

# Encode categorical features
x = pd.get_dummies(x)

# Split data into train and test sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=15)

# Extract the columns of the numerical features
numerical_columns = x_train.select_dtypes(include = ["float64", "int"]).columns

# Standardize the numerical features
ct = ColumnTransformer([("numerical", StandardScaler(), numerical_columns)], remainder="passthrough")
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

# Encode the labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [45]:
# Design the neural network model
model = Sequential()

# Add the input layer
model.add(InputLayer(shape = (x.shape[1],)))
model.add(Dense(12, activation = "relu"))
model.add(Dense(2, activation = "softmax"))



In [46]:
# Compile the model 
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer=Adam())

In [48]:
# Train the model
history = model.fit(x_train, y_train, epochs = 100, batch_size = 16, verbose = 1)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - accuracy: 0.8580 - loss: 0.3020
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - accuracy: 0.9053 - loss: 0.2581
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.9100 - loss: 0.2247
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step - accuracy: 0.8795 - loss: 0.2785
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.8867 - loss: 0.2463
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - accuracy: 0.8796 - loss: 0.2403
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.9006 - loss: 0.2495
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - accuracy: 0.8640 - loss: 0.2653
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.8840 - loss: 0.2742
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.8899 - loss: 0.2619
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step - accuracy: 0.9061 - loss: 0.2223
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0

In [73]:
# Evaluate the model 
loss, acc = model.evaluate(x_test, y_test)
print("Loss: ", loss, "Accuracy:", acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - accuracy: 0.7931 - loss: 0.5429
Loss:  0.5442015528678894 Accuracy: 0.7833333611488342


In [74]:
# Predict the outcome 

y_pred = model.predict(x_test, verbose=0)

# The selected code snippet, y_true = np.argmax(y_test, axis=1), is used to convert the one-hot encoded labels in the test set back to their original form.
# In the context of the provided code, y_test is a 2D array where each row represents a sample and each column corresponds to a class label. The values in the array are either 0 or 1, indicating whether the sample belongs to that class or not.
# The np.argmax function is used to find the index of the maximum value along the specified axis (in this case, axis=1). This operation returns an array where each element represents the class label with the highest probability for the corresponding sample in the y_test array.
# By assigning the result of np.argmax(y_test, axis=1) to y_true, we effectively convert the one-hot encoded labels back to their original form, which is necessary for evaluating the performance of the model using metrics such as accuracy or classification report.

y_true = np.argmax(y_test, axis=1)

In [78]:
# Classification report
print(classification_report(y_true, np.argmax(y_pred, axis = 1)))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85        43
           1       0.61      0.65      0.63        17

    accuracy                           0.78        60
   macro avg       0.73      0.74      0.74        60
weighted avg       0.79      0.78      0.79        60



In [79]:
np.argmax(y_pred, axis = 1)

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0], dtype=int64)

In [85]:
np.argmax(y_test, axis = 1)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0], dtype=int64)